In [1]:
import os

In [2]:
%pwd

'f:\\TB_Diagnosis_with_CNN_Classifier_Project\\TB_Diagnosis_with_CNN_Classifier_Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\TB_Diagnosis_with_CNN_Classifier_Project\\TB_Diagnosis_with_CNN_Classifier_Project'

In [5]:
import os

# Set the path to the folder containing the images
folder_path = "F:\\TB_Diagnosis_with_CNN_Classifier_Project\\TB_Diagnosis_with_CNN_Classifier_Project\\artifacts\\data_ingestion\\autistic"

# Set the prefix for the custom names
prefix = "autistic"

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Iterate through each file in the folder
for index, filename in enumerate(file_list):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust the file extensions as per your dataset
        # Generate the new custom name
        new_name = prefix + str(index + 1)  # Add 1 to the index since it starts from 0

        # Get the current file's path
        current_path = os.path.join(folder_path, filename)

        # Generate the new file's path
        new_path = os.path.join(folder_path, new_name + os.path.splitext(filename)[1])

        # Rename the file
        os.rename(current_path, new_path)

        print("Renamed:", filename, "to", new_name)

Renamed: autistic1.jpg to autistic1


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'F:\\TB_Diagnosis_with_CNN_Classifier_Project\\TB_Diagnosis_with_CNN_Classifier_Project\\artifacts\\data_ingestion\\autistic\\autistic10.jpg' -> 'F:\\TB_Diagnosis_with_CNN_Classifier_Project\\TB_Diagnosis_with_CNN_Classifier_Project\\artifacts\\data_ingestion\\autistic\\autistic2.jpg'

In [ ]:
import os

# Set the path to the folder containing the images
folder_path = "F:\\TB_Diagnosis_with_CNN_Classifier_Project\\TB_Diagnosis_with_CNN_Classifier_Project\\artifacts\\data_ingestion\\non_autistic"

# Set the prefix for the custom names
prefix = "non_autistic"

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Iterate through each file in the folder
for index, filename in enumerate(file_list):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust the file extensions as per your dataset
        # Generate the new custom name
        new_name = prefix + str(index + 1)  # Add 1 to the index since it starts from 0

        # Get the current file's path
        current_path = os.path.join(folder_path, filename)

        # Generate the new file's path
        new_path = os.path.join(folder_path, new_name + os.path.splitext(filename)[1])

        # Rename the file
        os.rename(current_path, new_path)

        print("Renamed:", filename, "to", new_name)

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate : float
    params_include_top : bool
    params_weights : str
    params_classes : int


In [7]:
from TBcnnClassifier.constants import *
from TBcnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config


    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)


    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

{'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://autisticdatabucket.s3.eu-north-1.amazonaws.com/consolidated.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}}
[2023-07-03 00:32:31,312: INFO: common: yaml file: config\config.yaml loaded successfully]
{'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 50, 'CLASSES': 2, 'WEIGHTS': 'imagenet', 'LEARNING_RATE': 0.01}
[2023-07-03 00:32:31,324: INFO: common: yaml file: params.yaml loaded successfully]
[2023-07-03 00:32:31,327: INFO: common: created directory at: artifacts]
[2023-07-03 00:32:31,331: INFO: common: created directory at: artifacts/prepare_base_model]
[2